In [7]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain


  Using cached langchainhub-0.1.20-py3-none-any.whl.metadata (659 bytes)
  Using cached chromadb-0.5.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached types_requests-2.32.0.20240602-py3-none-any.whl.metadata (1.8 kB)
  Using cached build-1.2.1-py3-none-any.whl.metadata (4.3 kB)
  Using cached posthog-3.5.0-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached opentelemetry_api-1.25.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.25.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached opentelemetry_instrumentation_fastapi-0.46b0-py3-none-any.whl.metadata (2.0 kB)
  Using cached opentelemetry_sdk-1.25.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached PyPika-0.48.9.tar.gz (67 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  P

In [15]:
!pip install faiss-cpu

   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.5 MB 220.2 kB/s eta 0:01:06
   ---------------------------------------- 0.0/14.5 MB 245.8 kB/s eta 0:00:59
   ---------------------------------------- 0.1/14.5 MB 547.6 kB/s eta 0:00:27
    --------------------------------------- 0.3/14.5 MB 1.1 MB/s eta 0:00:13
   - -------------------------------------- 0.4/14.5 MB 1.5 MB/s eta 0:00:10
   - -------------------------------------- 0.6/14.5 MB 1.7 MB/s eta 0:00:09
   -- ------------------------------------- 0.9/14.5 MB 2.3 MB/s eta 0:00:06
   --- ------------------------------------ 1.2/14.5 MB 2.8 MB/s eta 0:00:05
   --- ------------------------------------ 1.4/14.5 MB 3.0 MB/s eta 0:00:05
   ---- ----------------------------------- 1.7/14.5 MB 3.3 MB/s eta 0:00:04
   ----- -----

# RAG Overview

In [11]:
import os
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [9]:

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = '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'
os.environ['OPENAI_API_KEY'] = 'sk-proj-JE46Wf9LeJmeclm29rM4T3BlbkFJMB5L3A7daCynAQEIZpoK'


In [19]:
# Load Documents
loader = WebBaseLoader(
    web_paths=("https://mallahyari.github.io/rag-ebook/intro.html",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Embed
vectorstore = FAISS.from_texts(splits, embedding=OpenAIEmbeddings(model="text-embedding-ada-002"))

retriever = vectorstore.as_retriever()

#### RETRIEVAL and GENERATION ####

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatOpenAI(model="text-embedding-ada-002", temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
rag_chain.invoke("What is Task Decomposition?")



IndexError: list index out of range